# MHQA

## No Reasoning

In [ ]:
import json
import dotenv

with open("../../data/test_data.json", "r") as file:
    test_data = file.read()

test_data = json.loads(test_data)
dotenv.load_dotenv()
MODEL_NAME = "gemini-2.5-flash-preview-05-20"


In [ ]:
import os
import google.generativeai as genai
from google.api_core.client_options import ClientOptions

genai.configure(
    api_key=os.getenv("METIS_API_KEY"), 
    transport='rest',
    client_options=ClientOptions(api_endpoint="https://api.metisai.ir")
)

model = genai.GenerativeModel(
    MODEL_NAME,
    system_instruction="""
    Your task is to answer factual questions accurately and concisely, 
    using only the information you have learned. You will be shown examples of questions
      and their correct answers. Use them as a guide to understand how to answer similar questions.

    Do not explain your answer, list evidence, or include extra text. Only return the final answer.
    below are the answer types that you can use:
    اسامی عام
    شخص
    بلی/خیر
    تاریخ
    رویداد
    مکان
    اسامی خاص دیگر
    شماره
    کار هنری
    گروه یا سازمان
    صفت


    Examples:
    Q: در مقابل تبی که بنتونیت با نام گل ارمنی در آن معروف است چه چیز قرار دارد ؟
    <ANSWER> پزشکی مبتنی بر شواهد</ANSWER>

    Q: کدام یک از ستارگان فیلم زنی پشت پنجره بازیکن هاکی نیز می باشد ؟
    <ANSWER>وایـت راسل</ANSWER>

    Q: آیا سایتو دوسان یک نجیب زاده نظامی است ؟
    <ANSWER>بلی</ANSWER>

    Q: داده های ماهواره ای که نخستین‌ بار نقشه‌بردار موضوعی بر روی آن نصب شد در چه سالی به فضا پرتاب شد ؟
    <ANSWER>سال ۱۹۸۲</ANSWER>

    Q: جایزه ای که سیدیبه در سال 2007 آن را دریافت کرد متعلق به کدام جشنواره است ؟
    <ANSWER>جشنوارهٔ فیلم ونیز</ANSWER>

    Q: از بین کشورهای پرتغال و اسرائیل در کدام یک زودتر مسابقه آواز یوروویژن برگزار شد ؟
    <ANSWER>پرتغال</ANSWER>

    Q: کدام یک از زبان هایی که کلمه حمد در آن استفاده می شود دارای وضعیت رسمی در چندین ایالت هند نیز هست ؟
    <ANSWER>زبان اردو</ANSWER>

    Q: عدد اتمی عنصری که باعث میشه گاز نئون لامپ‌ها در صورت استفاده از آن رنگ آبی روشن تولید کنند چیست ؟
    <ANSWER>۸۰</ANSWER>

    Q: بازیگری امریکایی که در 23 ژوئیه 1967 بدنیا امد با بازی در چه فیلمی اولین بفتای خود را گرفت ؟
    <ANSWER>فیلم کاپوتی</ANSWER>

    Q: داده های ماهواره ای که نخستین‌ بار نقشه‌بردار موضوعی بر روی آن نصب شد توسط چه سازمانی مدیریت می شود ؟
    <ANSWER>ناسا</ANSWER>

    Q: تماشاگران کمپانی که استفنی مک‌من لوک از سال 2022 رئیس آن بود را چه کسانی تشکیل می دهند ؟
    <ANSWER>افراد جوان تا پیر</ANSWER>

    Q: نام دیگر قومی که پیروز یکم آن ها را به طور قطعی شکست داد چیست ؟
    <ANSWER>قوم کیدار</ANSWER>
    """
)


In [92]:
from tqdm import tqdm

results = []
for item in tqdm(test_data):
   question = item['question']
   answer = item['answer']
   _id = item['id']
   response = model.generate_content(
       question
   )
   model_answer = response.candidates[0].content.parts[0].text
   results.append({
       'question': question,
       'answer': answer,
       'model_answer': model_answer,
       'id': _id,
   })


100%|██████████| 152/152 [11:33<00:00,  4.56s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('results.csv', index=False)

## Evaluate

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import json
import re
from tqdm import tqdm

class AnswerEvaluation(BaseModel):
    is_correct: bool = Field(description="True if the model answer is semantically equivalent to the correct answer, False otherwise")

llm_judge = ChatOpenAI(
    model="gpt-4o-mini",  # Using gpt-4o-mini for better performance and cost efficiency
    temperature=0.1,  
    max_tokens=500,
    api_key=os.getenv("METIS_API_KEY"),
    base_url='https://api.metisai.ir/openai/v1',
).with_structured_output(AnswerEvaluation)

def clean_model_answer(model_answer: str) -> str:
    # Remove <ANSWER> tags
    cleaned = re.sub(r'<ANSWER>(.*?)</ANSWER>', r'\1', model_answer, flags=re.DOTALL)
    return cleaned.strip()

def evaluate_single_answer(question: str, correct_answer: str, model_answer: str) -> AnswerEvaluation:
    """Evaluate a single question-answer pair using LLM judge"""
    
    clean_model_answer_text = clean_model_answer(model_answer)
    
    # Create the evaluation prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert judge evaluating Persian/Farsi question-answer pairs. 
Your task is to determine if the model's answer is semantically equivalent to the correct answer.

Consider these factors:
- Semantic similarity (same meaning, different wording)
- Spelling variations and Persian writing differences
- Different but equivalent expressions (e.g., "بله" vs "بلی", both mean "yes")
- Context and cultural nuances in Persian language
- Both answers should convey the same factual information

Be strict but fair - minor spelling differences or equivalent expressions should be considered correct.
Only mark as incorrect if the meaning is genuinely different or wrong."""),
        
        ("human", """Question: {question}

Correct Answer: {correct_answer}
Model Answer: {model_answer}

Evaluate if the model answer is semantically equivalent to the correct answer.""")
    ])
    
    # Format the prompt with the actual values
    messages = prompt.format_messages(
        question=question,
        correct_answer=correct_answer,
        model_answer=clean_model_answer_text
    )
    
    try:
        evaluation = llm_judge.invoke(messages)
        return evaluation
    except Exception as e:
        return AnswerEvaluation(
            is_correct=False,
            reasoning=f"API call failed: {str(e)}"
        )

evaluated_results = []
for item in tqdm(results, desc="Evaluating answers"):
    
    question = item['question']
    correct_answer = item['answer']
    model_answer = item['model_answer']
    item_id = item['id']
    
    evaluation = evaluate_single_answer(question, correct_answer, model_answer)
    
    evaluated_item = {
        **item,
        'clean_model_answer': clean_model_answer(model_answer),
        'is_correct': evaluation.is_correct,
    }
    
    evaluated_results.append(evaluated_item)


df = pd.DataFrame(evaluated_results)
df.to_csv('evaluated_results.csv', index=False)



Evaluating answers:   0%|          | 0/152 [00:00<?, ?it/s]

Evaluating answers: 100%|██████████| 152/152 [02:19<00:00,  1.09it/s]


In [119]:
import numpy as np
len(df[df['is_correct'] == True])/ len(df['is_correct'])

0.5460526315789473

## with Reasoning

In [ ]:
import os
import google.generativeai as genai
from google.api_core.client_options import ClientOptions

genai.configure(
    api_key=os.getenv("METIS_API_KEY"), 
    transport='rest',
    client_options=ClientOptions(api_endpoint="https://api.metisai.ir")
)

model = genai.GenerativeModel(
    MODEL_NAME,
    system_instruction="""
    Your task is to answer factual questions accurately. For each question, first think step-by-step using only the knowledge you have learned. Write your reasoning inside a <Think>...</Think> block. Then, provide the final answer in a <ANSWER>...</ANSWER> block.

    Use the examples below as a guide. Only use information that would be reasonably known or inferred from training. Do not hallucinate or reference sources.

    Below are the answer types you may use (depending on the question):
    اسامی عام  
    شخص  
    بلی/خیر  
    تاریخ  
    رویداد  
    مکان  
    اسامی خاص دیگر  
    شماره  
    کار هنری  
    گروه یا سازمان  
    صفت  

    Examples:

    Q: در مقابل تبی که بنتونیت با نام گل ارمنی در آن معروف است چه چیز قرار دارد ؟  
    <Think>بنتونیت در طب سنتی شناخته می‌شود. در تقابل با طب سنتی معمولاً پزشکی مدرن یا پزشکی مبتنی بر شواهد قرار دارد.</Think>  
    <ANSWER>پزشکی مبتنی بر شواهد</ANSWER>

    Q: کدام یک از ستارگان فیلم زنی پشت پنجره بازیکن هاکی نیز می باشد ؟  
    <Think>در لیست بازیگران فیلم زنی پشت پنجره، وایت راسل حضور دارد. او همچنین بازیکن سابق هاکی است.</Think>  
    <ANSWER>وایـت راسل</ANSWER>

    Q: آیا سایتو دوسان یک نجیب زاده نظامی است ؟  
    <Think>سایتو دوسان یک سامورایی بوده و سامورایی‌ها نجیب‌زادگان نظامی در ژاپن هستند.</Think>  
    <ANSWER>بلی</ANSWER>

    Q: داده های ماهواره ای که نخستین‌ بار نقشه‌بردار موضوعی بر روی آن نصب شد در چه سالی به فضا پرتاب شد ؟  
    <Think>نقشه‌بردار موضوعی (TM) برای اولین بار بر روی ماهواره لندست ۴ نصب شد. این ماهواره در سال ۱۹۸۲ پرتاب شد.</Think>  
    <ANSWER>سال ۱۹۸۲</ANSWER>

    Q: جایزه ای که سیدیبه در سال 2007 آن را دریافت کرد متعلق به کدام جشنواره است ؟  
    <Think>سیدیبه در سال ۲۰۰۷ جایزه شیر طلایی دریافت کرد که متعلق به جشنواره فیلم ونیز است.</Think>  
    <ANSWER>جشنوارهٔ فیلم ونیز</ANSWER>

    Q: تماشاگران کمپانی که استفنی مک‌من لوک از سال 2022 رئیس آن بود را چه کسانی تشکیل می‌دهند ؟  
    <Think>استفنی مک‌من رئیس دبلیودبلیوئی بوده که برنامه‌هایش در کشورهای مختلف پخش می‌شود و طبق آمار مخاطبانش از رده سنی ۲ تا ۵۰ سال به بالا هستند.</Think>  
    <ANSWER>افراد جوان تا پیر</ANSWER>

    """
)


In [121]:
from tqdm import tqdm

results = []
for item in tqdm(test_data):
   question = item['question']
   answer = item['answer']
   _id = item['id']
   response = model.generate_content(
       question
   )
   model_answer = response.candidates[0].content.parts[0].text
   results.append({
       'question': question,
       'answer': answer,
       'model_answer': model_answer,
       'id': _id,
   })


100%|██████████| 152/152 [06:50<00:00,  2.70s/it]


In [ ]:
df = pd.DataFrame(results)
df.to_csv('results_reasoning.csv', index=False)

## Evaluate

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import json
import re
from tqdm import tqdm

class AnswerEvaluation(BaseModel):
    is_correct: bool = Field(description="True if the model answer is semantically equivalent to the correct answer, False otherwise")

llm_judge = ChatOpenAI(
    model="gpt-4o-mini",  # Using gpt-4o-mini for better performance and cost efficiency
    temperature=0.1,  # Low temperature for consistent judgments
    max_tokens=500,
    api_key=os.getenv("METIS_API_KEY"),
    base_url='https://api.metisai.ir/openai/v1',
).with_structured_output(AnswerEvaluation)

def clean_model_answer(model_answer: str) -> str:
    # Remove <ANSWER> tags
    """Clean the model answer by extracting text from <ANSWER> tags"""
    if not model_answer:
        return ""
    
    # Try to find <ANSWER> tags first
    answer_match = re.search(r'<ANSWER>(.*?)</ANSWER>', model_answer, re.DOTALL | re.IGNORECASE)
    if answer_match:
        return answer_match.group(1).strip()
    
    return model_answer.strip()

def evaluate_single_answer(question: str, correct_answer: str, model_answer: str) -> AnswerEvaluation:
    """Evaluate a single question-answer pair using LLM judge"""
    
    clean_model_answer_text = clean_model_answer(model_answer)
    
    # Create the evaluation prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert judge evaluating Persian/Farsi question-answer pairs. 
Your task is to determine if the model's answer is semantically equivalent to the correct answer.

Consider these factors:
- Semantic similarity (same meaning, different wording)
- Spelling variations and Persian writing differences
- Different but equivalent expressions (e.g., "بله" vs "بلی", both mean "yes")
- Context and cultural nuances in Persian language
- Both answers should convey the same factual information

Be strict but fair - minor spelling differences or equivalent expressions should be considered correct.
Only mark as incorrect if the meaning is genuinely different or wrong."""),
        
        ("human", """Question: {question}

Correct Answer: {correct_answer}
Model Answer: {model_answer}

Evaluate if the model answer is semantically equivalent to the correct answer.""")
    ])
    
    messages = prompt.format_messages(
        question=question,
        correct_answer=correct_answer,
        model_answer=clean_model_answer_text
    )
    
    try:
        evaluation = llm_judge.invoke(messages)
        return evaluation
    except Exception as e:
        return AnswerEvaluation(
            is_correct=False,
            reasoning=f"API call failed: {str(e)}"
        )

evaluated_results = []
for item in tqdm(results, desc="Evaluating answers"):
    
    question = item['question']
    correct_answer = item['answer']
    model_answer = item['model_answer']
    item_id = item['id']
    
    evaluation = evaluate_single_answer(question, correct_answer, model_answer)
    
    evaluated_item = {
        **item,
        'clean_model_answer': clean_model_answer(model_answer),
        'is_correct': evaluation.is_correct,
    }
    
    evaluated_results.append(evaluated_item)


df = pd.DataFrame(evaluated_results)
df.to_csv('evaluated_results_reasoning.csv', index=False)



Evaluating answers:   0%|          | 0/152 [00:00<?, ?it/s]

Evaluating answers: 100%|██████████| 152/152 [02:26<00:00,  1.04it/s]


# PQUAD

## No reasoning

In [ ]:
import os
import google.generativeai as genai
from google.api_core.client_options import ClientOptions
from tqdm import tqdm
genai.configure(
    api_key=os.getenv("METIS_API_KEY"), 
    transport='rest',
    client_options=ClientOptions(api_endpoint="https://api.metisai.ir")
)

model = genai.GenerativeModel(
    MODEL_NAME,
    system_instruction="""
    Your task is to answer factual questions accurately and concisely, 
    using only the information you have learned. You will be shown examples of questions
      and their correct answers. Use them as a guide to understand how to answer similar questions.

    Do not explain your answer, list evidence, or include extra text. Only return the final answer.

    Examples:
    Q: در مقابل تبی که بنتونیت با نام گل ارمنی در آن معروف است چه چیز قرار دارد ؟
    <ANSWER> پزشکی مبتنی بر شواهد</ANSWER>

    Q: کدام یک از ستارگان فیلم زنی پشت پنجره بازیکن هاکی نیز می باشد ؟
    <ANSWER>وایـت راسل</ANSWER>

    Q: آیا سایتو دوسان یک نجیب زاده نظامی است ؟
    <ANSWER>بلی</ANSWER>

    Q: داده های ماهواره ای که نخستین‌ بار نقشه‌بردار موضوعی بر روی آن نصب شد در چه سالی به فضا پرتاب شد ؟
    <ANSWER>سال ۱۹۸۲</ANSWER>

    Q: جایزه ای که سیدیبه در سال 2007 آن را دریافت کرد متعلق به کدام جشنواره است ؟
    <ANSWER>جشنوارهٔ فیلم ونیز</ANSWER>

    Q: از بین کشورهای پرتغال و اسرائیل در کدام یک زودتر مسابقه آواز یوروویژن برگزار شد ؟
    <ANSWER>پرتغال</ANSWER>

    Q: کدام یک از زبان هایی که کلمه حمد در آن استفاده می شود دارای وضعیت رسمی در چندین ایالت هند نیز هست ؟
    <ANSWER>زبان اردو</ANSWER>

    Q: عدد اتمی عنصری که باعث میشه گاز نئون لامپ‌ها در صورت استفاده از آن رنگ آبی روشن تولید کنند چیست ؟
    <ANSWER>۸۰</ANSWER>

    Q: بازیگری امریکایی که در 23 ژوئیه 1967 بدنیا امد با بازی در چه فیلمی اولین بفتای خود را گرفت ؟
    <ANSWER>فیلم کاپوتی</ANSWER>

    Q: داده های ماهواره ای که نخستین‌ بار نقشه‌بردار موضوعی بر روی آن نصب شد توسط چه سازمانی مدیریت می شود ؟
    <ANSWER>ناسا</ANSWER>

    Q: تماشاگران کمپانی که استفنی مک‌من لوک از سال 2022 رئیس آن بود را چه کسانی تشکیل می دهند ؟
    <ANSWER>افراد جوان تا پیر</ANSWER>

    Q: نام دیگر قومی که پیروز یکم آن ها را به طور قطعی شکست داد چیست ؟
    <ANSWER>قوم کیدار</ANSWER>
    """
)


In [ ]:
import pandas as pd
test_data = pd.read_csv('../../data/pquad/pquad_questions.csv', encoding='utf-8').to_dict(orient='records')

In [8]:


results = []

for item in tqdm(test_data):
    question = item['question']
    answer = item['answer']
    _id = item['id']
    context_id = item['context_id']
    response = model.generate_content(
        question
    )
    try:
        model_answer = response.candidates[0].content.parts[0].text
    except Exception as e:
        model_answer = "Error: " + str(e)
      
    results.append({
        'question': question,
        'answer': answer,
        'model_answer': model_answer,
        'id': _id,
        'context_id': context_id,
    })


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [19:11<00:00,  2.30s/it]


In [ ]:
df = pd.DataFrame(results)
df.to_csv('pquad_results.csv', index=False)

## Evaluate

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import json
import re
from tqdm import tqdm

class AnswerEvaluation(BaseModel):
    is_correct: bool = Field(description="True if the model answer is semantically equivalent to the correct answer, False otherwise")

llm_judge = ChatOpenAI(
    model="gpt-4o-mini",  
    temperature=0.1,  
    max_tokens=500,
    api_key=os.getenv("METIS_API_KEY"),
    base_url='https://api.metisai.ir/openai/v1',
).with_structured_output(AnswerEvaluation)

def clean_model_answer(model_answer: str) -> str:
    # Remove <ANSWER> tags
    cleaned = re.sub(r'<ANSWER>(.*?)</ANSWER>', r'\1', model_answer, flags=re.DOTALL)
    return cleaned.strip()

def evaluate_single_answer(question: str, correct_answer: str, model_answer: str) -> AnswerEvaluation:
    """Evaluate a single question-answer pair using LLM judge"""
    
    clean_model_answer_text = clean_model_answer(model_answer)
    
    # Create the evaluation prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert judge evaluating Persian/Farsi question-answer pairs. 
Your task is to determine if the model's answer is semantically equivalent to the correct answer.

Consider these factors:
- Semantic similarity (same meaning, different wording)
- Spelling variations and Persian writing differences
- Different but equivalent expressions (e.g., "بله" vs "بلی", both mean "yes")
- Context and cultural nuances in Persian language
- Both answers should convey the same factual information

Be strict but fair - minor spelling differences or equivalent expressions should be considered correct.
Only mark as incorrect if the meaning is genuinely different or wrong."""),
        
        ("human", """Question: {question}

Correct Answer: {correct_answer}
Model Answer: {model_answer}

Evaluate if the model answer is semantically equivalent to the correct answer.""")
    ])
    
    # Format the prompt with the actual values
    messages = prompt.format_messages(
        question=question,
        correct_answer=correct_answer,
        model_answer=clean_model_answer_text
    )
    
    try:
        evaluation = llm_judge.invoke(messages)
        return evaluation
    except Exception as e:
        return AnswerEvaluation(
            is_correct=False,
            reasoning=f"API call failed: {str(e)}"
        )

evaluated_results = []
for item in tqdm(results, desc="Evaluating answers"):
    
    question = item['question']
    correct_answer = item['answer']
    model_answer = item['model_answer']
    item_id = item['id']
    
    evaluation = evaluate_single_answer(question, correct_answer, model_answer)
    
    evaluated_item = {
        **item,
        'clean_model_answer': clean_model_answer(model_answer),
        'is_correct': evaluation.is_correct,
    }
    
    evaluated_results.append(evaluated_item)


df = pd.DataFrame(evaluated_results)
df.to_csv('pqaud_evaluated_results.csv', index=False)



Evaluating answers: 100%|██████████| 500/500 [07:49<00:00,  1.07it/s]


In [11]:
len(df[df['is_correct'] == True])/ len(df['is_correct'])

0.238

## With Reasoning

In [ ]:
import os
import google.generativeai as genai
from google.api_core.client_options import ClientOptions

genai.configure(
    api_key=os.getenv("METIS_API_KEY"), 
    transport='rest',
    client_options=ClientOptions(api_endpoint="https://api.metisai.ir")
)

model = genai.GenerativeModel(
    MODEL_NAME,
    system_instruction="""
    Your task is to answer factual questions accurately. For each question, first think step-by-step using only the knowledge you have learned. Write your reasoning inside a <Think>...</Think> block. Then, provide the final answer in a <ANSWER>...</ANSWER> block.

    Use the examples below as a guide. Only use information that would be reasonably known or inferred from training. Do not hallucinate or reference sources. 

    Examples:

    Q: در مقابل تبی که بنتونیت با نام گل ارمنی در آن معروف است چه چیز قرار دارد ؟  
    <Think>بنتونیت در طب سنتی شناخته می‌شود. در تقابل با طب سنتی معمولاً پزشکی مدرن یا پزشکی مبتنی بر شواهد قرار دارد.</Think>  
    <ANSWER>پزشکی مبتنی بر شواهد</ANSWER>

    Q: کدام یک از ستارگان فیلم زنی پشت پنجره بازیکن هاکی نیز می باشد ؟  
    <Think>در لیست بازیگران فیلم زنی پشت پنجره، وایت راسل حضور دارد. او همچنین بازیکن سابق هاکی است.</Think>  
    <ANSWER>وایـت راسل</ANSWER>

    Q: آیا سایتو دوسان یک نجیب زاده نظامی است ؟  
    <Think>سایتو دوسان یک سامورایی بوده و سامورایی‌ها نجیب‌زادگان نظامی در ژاپن هستند.</Think>  
    <ANSWER>بلی</ANSWER>

    Q: داده های ماهواره ای که نخستین‌ بار نقشه‌بردار موضوعی بر روی آن نصب شد در چه سالی به فضا پرتاب شد ؟  
    <Think>نقشه‌بردار موضوعی (TM) برای اولین بار بر روی ماهواره لندست ۴ نصب شد. این ماهواره در سال ۱۹۸۲ پرتاب شد.</Think>  
    <ANSWER>سال ۱۹۸۲</ANSWER>

    Q: جایزه ای که سیدیبه در سال 2007 آن را دریافت کرد متعلق به کدام جشنواره است ؟  
    <Think>سیدیبه در سال ۲۰۰۷ جایزه شیر طلایی دریافت کرد که متعلق به جشنواره فیلم ونیز است.</Think>  
    <ANSWER>جشنوارهٔ فیلم ونیز</ANSWER>

    Q: تماشاگران کمپانی که استفنی مک‌من لوک از سال 2022 رئیس آن بود را چه کسانی تشکیل می‌دهند ؟  
    <Think>استفنی مک‌من رئیس دبلیودبلیوئی بوده که برنامه‌هایش در کشورهای مختلف پخش می‌شود و طبق آمار مخاطبانش از رده سنی ۲ تا ۵۰ سال به بالا هستند.</Think>  
    <ANSWER>افراد جوان تا پیر</ANSWER>

    """
)


In [13]:
from tqdm import tqdm

results = []
for item in tqdm(test_data):
    question = item['question']
    answer = item['answer']
    _id = item['id']
    response = model.generate_content(
        question
    )
    try:
       model_answer = response.candidates[0].content.parts[0].text
    except Exception as e:
         model_answer = "Error: " + str(e)
    results.append({
        'question': question,
        'answer': answer,
        'model_answer': model_answer,
        'id': _id,
    })


100%|██████████| 500/500 [14:49<00:00,  1.78s/it]


In [ ]:
df = pd.DataFrame(results)
df.to_csv('pquad_results_reasoning.csv', index=False)

## Evaluate

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import json
import re
from tqdm import tqdm

class AnswerEvaluation(BaseModel):
    is_correct: bool = Field(description="True if the model answer is semantically equivalent to the correct answer, False otherwise")

llm_judge = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.1,  
    max_tokens=500,
    api_key=os.getenv("METIS_API_KEY"),
    base_url='https://api.metisai.ir/openai/v1',
).with_structured_output(AnswerEvaluation)

def clean_model_answer(model_answer: str) -> str:
    # Remove <ANSWER> tags
    """Clean the model answer by extracting text from <ANSWER> tags"""
    if not model_answer:
        return ""
    
    answer_match = re.search(r'<ANSWER>(.*?)</ANSWER>', model_answer, re.DOTALL | re.IGNORECASE)
    if answer_match:
        return answer_match.group(1).strip()
    
    return model_answer.strip()

def evaluate_single_answer(question: str, correct_answer: str, model_answer: str) -> AnswerEvaluation:
    """Evaluate a single question-answer pair using LLM judge"""
    
    clean_model_answer_text = clean_model_answer(model_answer)
    
    # Create the evaluation prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert judge evaluating Persian/Farsi question-answer pairs. 
Your task is to determine if the model's answer is semantically equivalent to the correct answer.

Consider these factors:
- Semantic similarity (same meaning, different wording)
- Spelling variations and Persian writing differences
- Different but equivalent expressions (e.g., "بله" vs "بلی", both mean "yes")
- Context and cultural nuances in Persian language
- Both answers should convey the same factual information

Be strict but fair - minor spelling differences or equivalent expressions should be considered correct.
Only mark as incorrect if the meaning is genuinely different or wrong."""),
        
        ("human", """Question: {question}

Correct Answer: {correct_answer}
Model Answer: {model_answer}

Evaluate if the model answer is semantically equivalent to the correct answer.""")
    ])
    
    # Format the prompt with the actual values
    messages = prompt.format_messages(
        question=question,
        correct_answer=correct_answer,
        model_answer=clean_model_answer_text
    )
    
    try:
        evaluation = llm_judge.invoke(messages)
        return evaluation
    except Exception as e:
        return AnswerEvaluation(
            is_correct=False,
            reasoning=f"API call failed: {str(e)}"
        )

evaluated_results = []
for item in tqdm(results, desc="Evaluating answers"):
    
    question = item['question']
    correct_answer = item['answer']
    model_answer = item['model_answer']
    item_id = item['id']
    
    evaluation = evaluate_single_answer(question, correct_answer, model_answer)
    
    evaluated_item = {
        **item,
        'clean_model_answer': clean_model_answer(model_answer),
        'is_correct': evaluation.is_correct,
    }
    
    evaluated_results.append(evaluated_item)


df = pd.DataFrame(evaluated_results)
df.to_csv('pquad_evaluated_results_reasoning.csv', index=False)



Evaluating answers: 100%|██████████| 500/500 [08:56<00:00,  1.07s/it]  


In [17]:
len(df[df['is_correct'] == True])/ len(df['is_correct'])

0.296